In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [37]:
qb_data = pd.read_csv('../data/nfl_ref/NFL QB Stats.csv')
qb_data.columns = qb_data.columns.str.lower()
adj_salaries = pd.read_csv('../data/salaries/QB_adjusted_salaries.csv')

In [38]:
qb_data[qb_data.player == 'Aaron Rodgers'].sample(3)

,year,player,pass yds,yds/att,att,cmp,cmp %,td,int,rate,1st,1st%,20+,40+,lng,sck,scky
348,2017,Aaron Rodgers,1675,7.0,238,154,64.7,16,6,97.2,85,35.7,21,4,72,22,168
672,2011,Aaron Rodgers,4643,9.2,502,343,68.3,45,6,122.5,208,41.4,64,13,93,36,219
505,2014,Aaron Rodgers,4381,8.4,520,341,65.6,38,5,112.2,219,42.1,59,15,80,28,174


In [57]:
adj_salaries[adj_salaries.player == 'Aaron Rodgers'].sample(3)

,player,year_signed,apy,med_adjusted_apy,mean_adjusted_apy,smooth_adjusted_apy
22,Aaron Rodgers,2018,33.500000,76.046189,4.342075,8.291695
29,Aaron Rodgers,2013,22.000000,50.416667,2.705836,3.984133
1,Aaron Rodgers,2022,50.271667,97.287066,6.728376,14.051800


In [ ]:
### based on above df structures, will join salaries on data that is prior to the year contract was signed. 

### so will join Aaron Rodgers 2018 contract information to all years 2013-2017

In [12]:
## removing rookie contracts from data 

rookie_years = adj_salaries.groupby('player')['year_signed'].min().reset_index()
rookie_years.columns = ['player', 'rookie_year']

adj_salaries = adj_salaries.merge(rookie_years, on='player')
adj_salaries = adj_salaries[adj_salaries['year_signed'] > adj_salaries['rookie_year']]

In [34]:
def next_salary(player_id, year, contract_df):

    ### making df of contracts happening after current year of play
    future_contracts = contract_df[(contract_df['player'] == player_id) &  
                                   (contract_df['year_signed'] > year)]


    ### getting contract signed most recently after current year of play, returning salary/yr
    if len(future_contracts) > 0:
        next_contract = future_contracts.sort_values('year_signed').iloc[0]
        return next_contract['apy']
    else:
        return np.nan 


In [ ]:
### applying next salary function to all rows in qb_data, joining the salary from their next contract 

qb_data['salary_per_year'] = qb_data.apply(
    lambda row: next_salary(row['player'], row['year'], adj_salaries), 
    axis=1
)

In [41]:
qb_data[qb_data.player == 'Aaron Rodgers'].head()

,year,player,pass yds,yds/att,att,cmp,cmp %,td,int,rate,1st,1st%,20+,40+,lng,sck,scky,salary_per_year
10,2022,Aaron Rodgers,3695,6.8,542,350,64.6,26,12,91.1,177,32.7,53,6,58,32,258,37.500000
84,2021,Aaron Rodgers,4115,7.8,531,366,68.9,37,4,111.9,213,40.1,55,10,75,30,188,50.271667
144,2020,Aaron Rodgers,4299,8.2,526,372,70.7,48,5,121.5,216,41.1,57,14,78,20,182,50.271667
214,2019,Aaron Rodgers,4002,7.0,569,353,62.0,26,4,95.4,189,33.2,52,12,74,36,284,50.271667
266,2018,Aaron Rodgers,4442,7.4,597,372,62.3,25,2,97.6,200,33.5,55,16,75,49,353,50.271667


In [54]:
qb_train_data = qb_data[qb_data.year <= 2021]
qb_train_data = qb_train_data.dropna(subset=['salary_per_year'])
qb_train_data.to_csv('../data/final/qb_train.csv', index=False)

In [55]:
qb_test_data = qb_data[qb_data.year > 2021]
qb_test_data = qb_test_data.dropna(subset=['salary_per_year'])
qb_test_data.to_csv('../data/final/qb_test.csv', index=False)

In [58]:
qb_test_data.sample(7)

,year,player,pass yds,yds/att,att,cmp,cmp %,td,int,rate,1st,1st%,20+,40+,lng,sck,scky,salary_per_year
57,2022,Anthony Brown,302,6.2,49,22,44.9,0,2,48.2,13,26.5,4,1,47,5,14,0.2160
13,2022,Derek Carr,3522,7.0,502,305,60.8,24,14,86.3,161,32.1,47,8,60,27,191,37.5000
32,2022,Zach Wilson,1688,7.0,242,132,54.6,6,7,72.8,73,30.2,26,6,79,23,175,6.0000
12,2022,Russell Wilson,3524,7.3,483,292,60.5,16,11,84.4,142,29.4,54,11,67,55,368,1.2100
25,2022,Justin Fields,2242,7.0,318,192,60.4,17,11,85.2,91,28.6,32,7,56,55,359,20.0000
70,2022,Chris Streveler,90,6.0,15,10,66.7,0,0,82.6,5,33.3,1,0,30,0,0,1.0100
64,2022,Trevor Siemian,184,7.1,26,15,57.7,1,1,76.4,7,26.9,4,0,33,2,14,1.3175
